In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.examples.tutorials.mnist import input_data
from random import randint

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(3, 28, 28, 1)


In [ ]:
def create_image(img_folder, create_img_folder):
    path_dir = '{}/'.format(img_folder)
    file_list = os.listdir(path_dir)
    
    for i in range(len(file_list)):
        path_sum = path_dir + file_list[i]
    
        img = cv2.imread('{}'.format(path_sum), cv2.IMREAD_GRAYSCALE) # gray processiong // 색깔이 달라도 괜찮음. 
        re_img = cv2.resize(img, (128,128)) # resize
        
        kernel = np.ones((3,3), np.uint8) # kernel
        erosion = cv2.erode(re_img, kernel, iterations = 1) # 더 가늘게, 음 근데 더 특징을 추출하는듯?
        #dilation= cv2.dilate(re_img, kernel, iterations = 1) # 더 두껍게

        rows, cols = re_img.shape[:2]
        # 이미지의 중심점을 기준으로 90도 회전 하면서 0.5배 Scale
        angle = [90,180,270]

        M1 = cv2.getRotationMatrix2D((cols/2, rows/2), angle[randint(0,2)], 1)
        M2 = np.float32([[1,0, randint(10,30)],[0,1, randint(10,30)]])

        M3 = cv2.getRotationMatrix2D((cols/2, rows/2), angle[randint(0,2)], 1)
        M4 = np.float32([[1,0,randint(10,30)],[0,1,randint(10,30)]])

        # basic
        dst1 = cv2.warpAffine(re_img, M1, (cols, rows))
        dst2 = cv2.warpAffine(re_img, M2, (cols, rows))

        # erosion
        dst3 = cv2.warpAffine(erosion, M3, (cols, rows))
        dst4 = cv2.warpAffine(erosion, M4, (cols, rows))
        
        a = file_list[i].split('.')
        b = a[0][:-1]
        
        # save!! 폴더 앞에 '/'를 붙이면 안 됨!!
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'1.jpg'), re_img) 
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'2.jpg'), dst1)
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'3.jpg'), dst2)
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'4.jpg'), erosion) 
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'5.jpg'), dst3) 
        cv2.imwrite('{}/{}'.format(create_img_folder, b+'6.jpg'), dst4) 

In [4]:
def make_image_data(image_folder):
    path_dir = '{}/'.format(image_folder)
    image_list = os.listdir(path_dir)

    img_batch = cv2.imread(path_dir + image_list[0])
    img_batch = np.expand_dims(img_batch, axis=0) #expand dimension

    for i in range(1, len(image_list)):
        img_add = cv2.imread(path_dir + image_list[i])
        img_add = np.expand_dims(img_add, axis=0)
        img_batch = np.concatenate([img_batch, img_add],axis=0)   

    img_batch = np.reshape(img_batch, (-1,128,128,3))  

In [5]:
def all_image_batch(image_folder):
    path_dir = '{}/'.format(image_folder)
    image_list = os.listdir(path_dir)
    img_batch = cv2.imread(path_dir + image_list[0])
    img_batch = np.expand_dims(img_batch, axis=0) # expand dimension
    
    for i in range(1, len(image_list)):
        img_add = cv2.imread(path_dir + image_list[i])
        img_add = np.expand_dims(img_add, axis=0)
        img_batch = np.concatenate([img_batch, img_add],axis=0)   

    return img_batch


def random_num_list(size):
    return_list = []
    path_dir = '2kind_train/' #변경해야하고,
    image_list = os.listdir(path_dir)
    
    for i in range(size):
        list_num = randint(0,len(image_list)-1) # 822 
        return_list.append(list_num)
    
    return return_list


def make_batch(all_image_np, label_np, batch_size):
    return_list = random_num_list(batch_size)
    img_batch = all_image_np[return_list]
    label_batch = label_np[return_list] 
    return img_batch, label_batch

In [6]:
# create label
def create_label(image_folder):
    label = []
    path_dir = '{}/'.format(image_folder)
    file_list = os.listdir(path_dir)
    model_num = 1 # 처음 하나 
    label_num = 0
    
    # model 개수
    for i in range(len(file_list)):
        if i > 0 and file_list[i-1].split('_')[0] != file_list[i].split('_')[0]:  
            model_num += 1 
    
    for i in range(len(file_list)):
        if i > 0 and file_list[i-1].split('_')[0] != file_list[i].split('_')[0]:  
            label_num += 1 
        each_label = model_num * [0]
        each_label[label_num] = 1
        label.append(each_label)
    
    label = np.array(label)
    return label


all_label = create_label('2kind_train')
all_image = all_image_batch('2kind_train')

In [7]:
train_image, train_label = make_batch(all_image, all_label, 200)
test_image, test_label = make_batch(all_image, all_label, 100)

In [8]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)
    print('1_', l1.shape)
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)
    print('2_', l2.shape)
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    print('3_', l3.shape)
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    print('4_', l4.shape)
    pyx = tf.matmul(l4, w_o)
    print('5_', pyx)
    return pyx

X = tf.placeholder("float", [None, 128, 128, 3])
Y = tf.placeholder("float", [None, 2])

w = init_weights([3, 3, 3, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4* 16, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 2])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)
print(py_x.shape, 'py_X')
aaa = np.float32(np.array([[1,0],[1,0],[1,0]]))
print(aaa.shape)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = py_x, labels= Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    print('Start')
    for i in range(100):

        sess.run(train_op, feed_dict={X: train_image, Y: train_label,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})


        print(i, np.mean(np.argmax(test_label, axis=1) ==
                         sess.run(predict_op, feed_dict={X: test_image,
                                                         Y: test_label,
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))


1_ (?, 64, 64, 32)
2_ (?, 32, 32, 64)
3_ (?, 32768)
4_ (?, 625)
5_ Tensor("MatMul_1:0", shape=(?, 2), dtype=float32)
(?, 2) py_X
(3, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Start
0 0.63
1 0.63
2 0.63
3 0.63
4 0.63
5 0.63
6 0.63
7 0.63
8 0.63
9 0.63
10 0.63
11 0.63
12 0.63
13 0.63
14 0.63
15 0.63
16 0.63
17 0.63
18 0.63
19 0.63
20 0.63
21 0.63
22 0.63
23 0.63
24 0.63
25 0.63
26 0.63
27 0.63
28 0.63
29 0.63
30 0.63
31 0.63
32 0.63
33 0.63
34 0.63
35 0.63


KeyboardInterrupt: 